# Datos

http://www.santafe.gob.ar/ms/covid19/datosabiertos/

In [35]:
from datetime import datetime

import pandas as pd

from plotly import express as px

pd.set_option('display.max_rows', 1000)

In [36]:
casos = pd.read_csv("./query_result_2021-03-29T21_07_34.265785-03_00.csv")
casos.rename(columns={col: col.lower().replace(" ", "_") for col in casos.columns}, inplace=True)
casos.rename(columns={f: f + "_accum" for f in ("confirmados", "descartados", "en_estudio", "notificaciones")},
             inplace=True)
casos.localidad = casos.localidad.str.capitalize()
casos.fecha = pd.to_datetime(casos.fecha)
casos.sort_values("fecha", inplace=True)

# agregar delta y medias moviles de casos por localidad
for localidad in casos.localidad.unique():
    index_localidad = casos.localidad == localidad
    casos_localidad = casos[index_localidad]
    
    confirmados_localidad = casos_localidad["confirmados_accum"] - casos_localidad["confirmados_accum"].shift(1)
    media_3_localidad = confirmados_localidad.rolling(3).mean()
    media_7_localidad = confirmados_localidad.rolling(7).mean()
    
    casos.loc[index_localidad, "confirmados"] = confirmados_localidad
    casos.loc[index_localidad, "confirmados_media_3"] = media_3_localidad
    casos.loc[index_localidad, "confirmados_media_7"] = media_7_localidad

casos.tail()

,fecha,departamento,localidad,confirmados_accum,descartados_accum,en_estudio_accum,notificaciones_accum,confirmados,confirmados_media_3,confirmados_media_7
287,2021-03-28 00:00:00-03:00,Rosario,Soldini,434,309,28,771,0.0,0.333333,0.571429
288,2021-03-28 00:00:00-03:00,Rosario,Villa amelia,75,90,9,174,0.0,0.333333,0.142857
289,2021-03-28 00:00:00-03:00,Rosario,Villa gobernador galvez,5456,6036,355,11847,3.0,6.666667,6.857143
291,2021-03-28 00:00:00-03:00,San Cristóbal,Aguara grande,4,3,0,7,0.0,0.000000,0.000000
0,2021-03-28 00:00:00-03:00,9 de Julio,Antonio pini,3,0,0,3,0.0,0.000000,0.000000


In [93]:
def graficar(casos, campo="confirmados_media_7", por_localidad=False, region=None):   
    if region is None:
        region = ', '.join(casos.localidad.unique())
    
    aclaracion = ""
    if "media" in campo:
        aclaracion = f" (media móvil de {campo.split('_')[-1]} días)"
    
    if por_localidad:
        casos = casos.groupby(["fecha", "localidad"]).sum().reset_index().set_index("fecha")
        color = "localidad"
    else:
        casos = casos.groupby("fecha").sum()
        color = None
   
    labels = {
        campo: "",
        "fecha": "",
    }

    fig = px.line(
        casos,
        y=campo, 
        title=f"Casos confirmados en {region}{aclaracion}",
        labels=labels,
        color=color,
    )

    fig.add_vline(
        x=datetime(2021, 2, 17), 
        line_width=2, 
        line_dash="dot", 
        line_color="green",
    )
    
    return fig

In [94]:
graficar(casos[casos.localidad == "Rafaela"])

In [95]:
graficar(casos[casos.localidad.isin({"Rafaela", "Sunchales"})], por_localidad=True)

In [96]:
graficar(casos[casos.localidad.isin({"Rafaela", "Sunchales"})], por_localidad=False)

In [97]:
graficar(casos, por_localidad=False, region="Provincia de Santa Fe")